<a href="https://colab.research.google.com/github/ChanMunFai/aifairness/blob/main/MF_2022_IC_ai_fairness_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for the Tutorial session of the Machine Learning module of the Ethics, Privacy and AI in Society course.

In this notebook, we will train a simple classifier on a dataset that has an internal bias in its datapoints. We will use the **AI Fairness 360** library to implement a de-biasing method, and also to observe the performance of our classifier using a series of metrics, that go beyond pure accuracy.  

Install the library and download the dataset.

In [ ]:
!pip install 'aif360[LFR]'
!pip install fairlearn

In [5]:
cd /usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult

/usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult


In [6]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

--2022-02-04 09:00:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/x-httpd-php]
Saving to: ‘adult.data’

adult.data          100%[===================>]   3.79M  7.31MB/s    in 0.5s    

2022-02-04 09:00:46 (7.31 MB/s) - ‘adult.data’ saved [3974305/3974305]

--2022-02-04 09:00:46--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5229 (5.1K) [application/x-httpd-php]
Saving to: ‘adult.names’

adult.names         100%[===================>]   5.11K  --.-KB/s    in 0s      

2022-

In [120]:
#STEP 1: Import the libraries and set the random seed.

import numpy as np

from aif360.datasets import StructuredDataset, BinaryLabelDataset
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing

import pdb

from sklearn.preprocessing import StandardScaler  #MinMaxScaler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import *

np.random.seed(0)

In [206]:
#STEP 2: We define where's the bias in the features of our dataset.

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])


#STEP 3: We split between training and test set.
train, test = dataset_orig.split([0.7], shuffle=True)
print("training data size", train.features.shape)
print("dataset feature names", train.feature_names)

#Normalize the dataset, both train and test. This should always be done in any machine learning pipeline!
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(train.features)
y_train = train.labels.ravel()

untransformed_X = scale_orig.inverse_transform(X_train)

X_test = scale_orig.fit_transform(test.features)
y_test = test.labels.ravel()

print(untransformed_X)

training data size (34189, 18)
dataset feature names ['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']
[[1.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [1.11022302e-16 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 ...
 [1.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [203]:
print(test)

               instance weights features  ...                     labels
                                          ...                           
                                    race  ... Education Years=>12       
instance names                            ...                           
723                         1.0      0.0  ...                 0.0    0.0
25734                       1.0      1.0  ...                 1.0    1.0
18152                       1.0      0.0  ...                 0.0    0.0
20053                       1.0      1.0  ...                 0.0    0.0
264                         1.0      1.0  ...                 1.0    0.0
...                         ...      ...  ...                 ...    ...
42249                       1.0      0.0  ...                 0.0    0.0
37193                       1.0      1.0  ...                 0.0    0.0
12260                       1.0      1.0  ...                 1.0    0.0
47789                       1.0      1.0  ...      

In [97]:
print(dir(test))
print(test.feature_names)
print(test.labels)

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_de_dummy_code_df', '_parse_feature_names', 'align_datasets', 'convert_to_dataframe', 'copy', 'export_dataset', 'favorable_label', 'feature_names', 'features', 'ignore_fields', 'import_dataset', 'instance_names', 'instance_weights', 'label_names', 'labels', 'metadata', 'privileged_protected_attributes', 'protected_attribute_names', 'protected_attributes', 'scores', 'split', 'subset', 'temporarily_ignore', 'unfavorable_label', 'unprivileged_protected_attributes', 'validate_dataset']
['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (dec

In [8]:
#STEP 4: Train a standard classifier, compute accuracy and fairness metrics.
#We use a simple Logistic Regression to parametrise our classifier. 
#You can try different classifiers and hyperparameters, checking how the metrics will change.

learner = LogisticRegression(solver='liblinear', random_state=1)  #(C=reg_best, solver='liblinear', random_state=1) 
learner.fit(X_train,y_train)
predictions = learner.predict(X_test)

test_pred = test.copy()
test_pred.labels = predictions

print("Accuracy", sum(predictions==y_test)/len(y_test))

#This is the set of metrics we use, taken from https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html.
#In different forms, they all measure bias. 
metric = ClassificationMetric(test, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
metric_arrs = {}
#Statistical Parity Difference measures the difference of the above values instead of ratios, hence we
#would like it to be close to 0.
metric_arrs['stat_par_diff']=(metric.statistical_parity_difference())
#Equal opportunity difference measures the ability of the classifier to accurately classify a datapoint as positive
#regardless of the presence of the unpriviliged feature. We would like it to be close to 0. A negative value signals bias
#towards priviliged.
metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
#Average of difference in FPR and TPR for unprivileged and privileged groups. A value of 0 indicates equality of odds.
metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
#Balanced accuracy is a general metric, not dependent on bias. We would like to have it close to 1, meaning 
#that the classifier can equally detect positive and negative classes.
metric_arrs['bal_acc']=((metric.true_positive_rate() + metric.true_negative_rate()) / 2)
#We would like Disparate Impact to be close to 1. It measures the ratio between the likelihood of the class being
#predicted as positive if we have the unpriviliged feature and the the same likelihood with the priviliged feature.
#Values close to 0 indicate strong bias.
metric_arrs['disp_imp']=(metric.disparate_impact())
print(metric_arrs)


Accuracy 0.8042039172865625
{'stat_par_diff': -0.20557244174265452, 'eq_opp_diff': -0.4414141414141414, 'avg_odds_diff': -0.27273605621431707, 'bal_acc': 0.657262071666589, 'disp_imp': 0.0}


Take a look at the metrics. Do you see a bias, given the metrics we used?
Let's use **Reweighing**, a method to tackle dataset bias by applying a weight to the training datapoints, so that some are considered more than others in the computation of the loss function.

In [ ]:

#STEP 5: Mitigate the bias, e.g. by transforming the original dataset via reweighing.
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
#We obtain a set of weights for the training set, to use in scikit-learn.
train = RW.fit_transform(train)

print("subgroup weights", np.unique(train.instance_weights))

#We use the same classifier as before, but now we use the instance weights in the training phase.
learner = LogisticRegression(solver='liblinear', random_state=1)  #(C=reg_best, solver='liblinear', random_state=1) 
learner.fit(X_train,y_train,sample_weight=train.instance_weights)
predictions = learner.predict(X_test)
print("Accuracy", sum(predictions==y_test)/len(y_test))


test_pred = test.copy()
predictions.resize((len(predictions),1))
test_pred.labels = predictions

metric = ClassificationMetric(test, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
metric_arrs = {}
metric_arrs['stat_par_diff']=(metric.statistical_parity_difference())
metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
metric_arrs['bal_acc']=((metric.true_positive_rate()                             + metric.true_negative_rate()) / 2)
metric_arrs['disp_imp']=(metric.disparate_impact())
print(metric_arrs)

#pdb.set_trace()

subgroup weights [0.78875735 0.85514075 1.09270553 2.1493453 ]
Accuracy 0.7905548351873336
{'stat_par_diff': -0.05772377304710344, 'eq_opp_diff': 0.03513180586351322, 'avg_odds_diff': 0.019935709638750347, 'bal_acc': 0.6671783946216994, 'disp_imp': 0.706625314122085}


Take another look at the metrics. Is the situation better in terms of bias? What about the accuracy?

(**TASK 1**) For each of the subgroup (combinations of YxS), build your code to compute the weights based on lecture slides. Verify the weights are the same as when using the library. 

(**TASK 2**) On the accuracy and generalisation trade-off: Change the regularisation strength via a hyperparameter setting. In logistic regression, use:`LogisticRegression(C=C_value) `.
How does this change the accuracy and fairness metrics? 

(**TASK 3**) Using train data, perform 5-fold cross validation; by varying the trade-off hyperparameter, select the model with the highest accuracy, and evaluate it on the test data (**assignment**).

(**TASK 4**) Learn about different pre-, in-, or post-processing methods and how to evaluate them using the library. Perform empirical analysis on accuracy and fairness trade-off (**assignment**).



## Re-weighing

Compute weights for all combinations of Y (outcome) and A(sensitive attribute)

Intuition is to force Y and A to be independent.


### Task 2

In [161]:
learner = LogisticRegression(solver='liblinear', random_state=1, C = 0.01, penalty = "l2")  #(C=reg_best, solver='liblinear', random_state=1) 
learner.fit(X_train,y_train)
predictions = learner.predict(X_test)

test_pred = test.copy()
test_pred.labels = predictions

print("Accuracy", sum(predictions==y_test)/len(y_test))

metric = ClassificationMetric(test, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
metric_arrs = {}
metric_arrs['stat_par_diff']=(metric.statistical_parity_difference())
metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
metric_arrs['bal_acc']=((metric.true_positive_rate() + metric.true_negative_rate()) / 2)
metric_arrs['disp_imp']=(metric.disparate_impact())
print(metric_arrs)


Accuracy 0.8100730225892309
{'stat_par_diff': -0.21391963919639195, 'eq_opp_diff': -0.47379100439634764, 'avg_odds_diff': -0.2873440975798476, 'bal_acc': 0.6695087466257853, 'disp_imp': 0.0}


In [164]:
print(test)
print(test_pred)

               instance weights features  ...                     labels
                                          ...                           
                                    race  ... Education Years=>12       
instance names                            ...                           
723                         1.0      0.0  ...                 0.0    0.0
25734                       1.0      1.0  ...                 1.0    1.0
18152                       1.0      0.0  ...                 0.0    0.0
20053                       1.0      1.0  ...                 0.0    0.0
264                         1.0      1.0  ...                 1.0    0.0
...                         ...      ...  ...                 ...    ...
42249                       1.0      0.0  ...                 0.0    0.0
37193                       1.0      1.0  ...                 0.0    0.0
12260                       1.0      1.0  ...                 1.0    0.0
47789                       1.0      1.0  ...      

ValueError: ignored

---
## Task 3

Link for Cross-Validation: https://scikit-learn.org/stable/modules/cross_validation.html
- Use cross_validate()

Link to use your own score for CV: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter


It seemes easier to use numpy for CV instead, so let me do that

In [105]:
# print(test.feature_names)
column_names = test.feature_names.copy()
print(column_names)
# print(test.label_names)

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


In [303]:
from pandas.core.internals.blocks import external_values
import pandas as pd
from aif360.metrics import DatasetMetric

def k_fold_cross_validation(X, y, k, model, feature_names, unprivileged_groups = None, privileged_groups = None): 
  fold_indices = np.arange(X.shape[0])
  np.random.shuffle(fold_indices)

  eval_indices = np.array_split(fold_indices, k)

  metrics = {}
  accuracy_list = []
  feature_names = feature_names.copy()
  feature_names.extend(["Label"])
  
  # k-fold cross validation 
  for e in eval_indices:
    # print(len(e))

    eval_set_X = X[e]
    eval_set_X = scale_orig.inverse_transform(eval_set_X).round(1)
    eval_set_Y = y[e]
    mask_eval = np.ones(X.shape[0], bool)
    mask_eval[e] = False

    train_set_X = X[mask_eval]
    train_set_Y = y[mask_eval]
    # print(eval_set_X.shape, train_set_X.shape)

    model.fit(train_set_X, train_set_Y)

    predictions = model.predict(eval_set_X)
    accuracy = sum(predictions == eval_set_Y)/len(eval_set_Y)
    accuracy_list.append(accuracy)

    # Use AIFairness360 metrics 
    eval = np.concatenate((eval_set_X, eval_set_Y.reshape(-1, 1)), axis = 1)
    eval = pd.DataFrame(eval)
    eval = eval.astype(int)
    
    eval.columns = feature_names

    test_pred = eval.copy()
    test_pred.iloc[:, -1] = predictions

    eval = BinaryLabelDataset(favorable_label=1,unfavorable_label=0,
                              df=eval,label_names=['Label'],
                              protected_attribute_names=["race", "sex"])
    
    test_pred = BinaryLabelDataset(favorable_label=1,unfavorable_label=0,
                              df=test_pred,label_names=['Label'],
                              protected_attribute_names=["race", "sex"])
    
    print(eval)
    # print(eval.unprivileged_protected_attributes)
    
    # test_pred = eval.copy()
    # test_pred.labels= predictions
    print(test_pred)

    dm = DatasetMetric(eval, unprivileged_groups, privileged_groups)
    print(dm.num_instances(True))
    print(dm.num_instances(False))

  
    metric = ClassificationMetric(eval, test_pred, unprivileged_groups, privileged_groups)
    print(metric.accuracy())
    print(metric.base_rate())
    print(metric.base_rate(True))
    print(metric.average_odds_difference())
    print(metric.statistical_parity_difference())
    print(metric.equal_opportunity_difference())
    print(metric.average_odds_difference())
    print((metric.true_positive_rate() + metric.true_negative_rate()) / 2)
    print(metric.disparate_impact())

    print("\n")


  mean_accuracy = np.mean(accuracy_list)
  accuracy_list.append(mean_accuracy)

  metrics["Accuracy"] = accuracy_list

  return eval


In [302]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
logistic_regression = LogisticRegression(solver='liblinear', random_state=1, C = 0.01, penalty = "l2") 
eval_a = k_fold_cross_validation(X_train, y_train, k = 2, model = logistic_regression, feature_names = column_names, 
                        unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

               instance weights            features  ...                     labels
                                protected attribute  ...                           
                                               race  ... Education Years=>12       
instance names                                       ...                           
0                           1.0                 1.0  ...                 0.0    0.0
1                           1.0                 1.0  ...                 0.0    0.0
2                           1.0                 1.0  ...                 0.0    0.0
3                           1.0                 0.0  ...                 0.0    0.0
4                           1.0                 1.0  ...                 1.0    1.0
...                         ...                 ...  ...                 ...    ...
17090                       1.0                 1.0  ...                 1.0    1.0
17091                       1.0                 1.0  ...                 1.0

invalid value encountered in double_scalars
invalid value encountered in double_scalars


               instance weights            features  ...                     labels
                                protected attribute  ...                           
                                               race  ... Education Years=>12       
instance names                                       ...                           
0                           1.0                 1.0  ...                 0.0    0.0
1                           1.0                 1.0  ...                 0.0    0.0
2                           1.0                 0.0  ...                 1.0    0.0
3                           1.0                 1.0  ...                 0.0    0.0
4                           1.0                 1.0  ...                 0.0    0.0
...                         ...                 ...  ...                 ...    ...
17089                       1.0                 1.0  ...                 1.0    0.0
17090                       1.0                 1.0  ...                 1.0

invalid value encountered in double_scalars
invalid value encountered in double_scalars


In [218]:
print(test.features)

[[1. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 0. ... 0. 0. 0.]]


---

In [230]:
print(eval_a.protected_attributes)
print(eval_a.protected_attribute_names)
print(eval_a.privileged_protected_attributes)
print(eval_a.features)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
['race', 'sex']
[array([0.]), array([0.])]
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
for i in eval_a.protected_attributes:
  print(i)

In [ ]:
from sklearn.model_selection import cross_validate
scoring = ['accuracy']
learner = LogisticRegression(solver='liblinear', random_state=1, C = 0.01, penalty = "l2") 
scores = cross_validate(learner, X_train, y_train, scoring=scoring, cv = 5)

In [ ]:
scores

{'fit_time': array([0.12234139, 0.19997263, 0.14443493, 0.15501308, 0.12455058]),
 'score_time': array([0.00662827, 0.00860167, 0.00227618, 0.00251174, 0.0027256 ]),
 'test_accuracy': array([0.79789412, 0.80242761, 0.80783855, 0.80535244, 0.8067866 ])}

In [ ]:
values_C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

scores_array = {}
for value in values_C: 
  scoring = ['accuracy']
  learner = LogisticRegression(solver='liblinear', random_state=1, C = value, penalty = "l2") 
  scores = cross_validate(learner, X_train, y_train, scoring=scoring, cv = 5)
  average_accuracy = np.mean(scores['test_accuracy'])
  print(average_accuracy)
  scores_array[value] = np.append(scores['test_accuracy'], average_accuracy)

# print(scores_array)

0.7972155789282572
0.8033579011506093
0.8040598650658761
0.8041476100205415
0.8041476100205415
0.8041476100205415
0.8041476100205415
0.8041476100205415


In [ ]:
scores_array

{0.0001: array([0.79160573, 0.79540801, 0.80257385, 0.79584674, 0.80064356,
        0.79721558]),
 0.001: array([0.79789412, 0.80111144, 0.80754607, 0.80359754, 0.80664034,
        0.8033579 ]),
 0.01: array([0.79789412, 0.80242761, 0.80783855, 0.80535244, 0.8067866 ,
        0.80405987]),
 0.1: array([0.79833285, 0.80242761, 0.80783855, 0.80535244, 0.8067866 ,
        0.80414761]),
 1: array([0.79833285, 0.80242761, 0.80783855, 0.80535244, 0.8067866 ,
        0.80414761]),
 10: array([0.79833285, 0.80242761, 0.80783855, 0.80535244, 0.8067866 ,
        0.80414761]),
 100: array([0.79833285, 0.80242761, 0.80783855, 0.80535244, 0.8067866 ,
        0.80414761]),
 1000: array([0.79833285, 0.80242761, 0.80783855, 0.80535244, 0.8067866 ,
        0.80414761])}

Now, I want to also add in my other metrics in CV. Let's see if they work. 

In [ ]:
metric = ClassificationMetric(test, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
metric.statistical_parity_difference()

-0.20557244174265452

In [ ]:
from sklearn.metrics import make_scorer

def statistical_parity_fn(y_true, y_pred):
  metric = ClassificationMetric(y_true, y_pred, unprivileged_groups, privileged_groups)
  return metric.statistical_parity_difference()

statistical_parity = make_scorer(statistical_parity_fn)

In [ ]:
values_C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

scores_array = {}
for value in values_C: 
  # scoring = ['accuracy', statistical_parity]
  scoring = {'accuracy': 'accuracy',
            'statistical parity': statistical_parity}
  learner = LogisticRegression(solver='liblinear', random_state=1, C = value, penalty = "l2") 
  scores = cross_validate(learner, X_train, y_train, scoring=scoring, cv = 5)
  average_accuracy = np.mean(scores['test_accuracy'])
  print(average_accuracy)
  scores_array[value] = np.append(scores['test_accuracy'], average_accuracy)

# print(scores_array)